In [2]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [7]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [8]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [9]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [10]:
delete_data = pd.DataFrame(columns=['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value'])
delete_data.to_csv('DELETE.csv', index=False)

data_element_group = 'VPHRxGDLoLM'
elements = api.get(f'dataElementGroups/{data_element_group}', params={'fields':'dataElements'})
elements = elements.json()['dataElements']

for org_unit in org_units:
    
    metadatas = []

    for element in elements:
        element_id = element['id']
        indicators = api.get('dataElementOperands', params={'paging':False, 'fields':'id,name', 'filter':f'dataElement.id:eq:{element_id}'})
        indicators = indicators.json()['dataElementOperands']

        for indicator in indicators:
            data = {'dataElement': indicator['id'].split('.')[0], 'period': period, 'orgUnit':org_unit['id'], 'categoryOptionCombo':indicator['id'].split('.')[1]}
            metadatas.append(data)

    metadatas = pd.json_normalize(metadatas)

    delete_data = pd.read_csv('DELETE.csv')
    delete_data = pd.concat([delete_data, metadatas])
    delete_data.to_csv('DELETE.csv', index=False, encoding='utf-8')

In [4]:
delete = pd.read_csv('DELETE.csv')
delete = delete[delete['dataElement'] == 'bII4eG3osk5']

In [7]:
delete['period'] = '2024Q4'

In [9]:
delete.to_csv('/Users/steliomo/Documents/ANGOLA_TRACKER/2025Q1/V2/PMTCT_DELETE.csv', index=False)